In [70]:
import numpy as np
import tensorflow
import random
import string
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dense, SimpleRNN
from tensorflow.keras.preprocessing import sequence

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [73]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 20000)

In [74]:
sents[0:10]

['  درميان سياستمداران فرانسه درمورد عضويت ترکيه دراتحاديه اروپا که متشکل از25 کشوراست، تشتت آرا واختلاف نظرشديدی وجود دارد',
 '  ترکيه که يک کشور اسلامی است مدتهاست که با اسرائيل مناسبات نزديک نظامی و تجاری دارد',
 ' مقامات گفتند بمب انداز اتومبیل خود را به میان جمعیت راند و بعد آنرا منفجر کرد',
 '  در ناحیه دیگری در امتداد مرز افغانستان، مقامات می گویند نیروهای پاکستانی از روز جمعه حد اقل ۱۶ ستیزه گر را در منطقه قبیله ای باجور کشته اند',
 ' محمد امين فاطمی روز چهارشنبه در کابل به خبرنگاران گفت : مقتولين برای تحويل دارو و تجهيرات پزشکی عازم چندين کلينيک در ولايت هلمند بودند، و اجساد آنها شامگاه سه شنبه در يک ناحيه صحرائی پيدا شد',
 ' ویلیام مک کینلی، تصمیم گیری در مورد این مساله مالی را به کنگره ارجاع کرد و از کنگره خواست جلسات ویژه ای در این رابطه برگزار کند',
 ' هواپيمای بزرگ مسافری که بيش از يک سال در فروگاه لوآناد در کشور آنگولا پارک شده بود قرار نبود جائی برود',
 '  در تنيس زنان، آميلی مورسمو بازيکن فرانسوی، اميددار شماره دوم است',
 ' پليس عراق نيز می گويد مردان مسلح ۲۲ چوپان را 

In [75]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
from hazm import *
normalizer = Normalizer()

In [77]:
sents_norm = [normalizer.normalize(s) for s in sents]
sents_lengths = [len(s) for s in sents_norm]
sents_short = []

for s in sents_norm:
  if len(s) < 100:
    sents_short.append(s)

sents_short

[' ترکیه که یک کشور اسلامی است مدتهاست که با اسرائیل مناسبات نزدیک نظامی و تجاری دارد',
 ' مقامات گفتند بمب انداز اتومبیل خود را به میان جمعیت راند و بعد آنرا منفجر کرد',
 ' در تنیس زنان، آمیلی مورسمو بازیکن فرانسوی، امیددار شماره دوم است',
 ' پلیس عراق نیز می\u200cگوید مردان مسلح ۲۲ چوپان را درنزدیکی شهرکربلا ربودند',
 ' کشورهای دانمارک و نروژ از شهروندان خود مصرانه خواستند خاک سوریه را ترک کنند',
 ' ژنرال مشرف به نقل از آقای آرمیتیج گفت: " آماده بمباران باشید',
 ' درخواست جدیدی در روز پنجشنبه فقط ۱۶ میلیون دلار جمع آوری کرد',
 ' پلیس همچنین میگوید توقیف\u200cهای شهر منچستر با توقیف\u200cهای لندن ارتباطی نداشته است',
 ' هزاران نفر کارکنان شرکت برق جزو کسانی بودند که امروز در بغداد حقوق گرفتند',
 ' بنظر می\u200cرسد که احمد قدیرف، مدیر اجرائی منتخب دولت روسیه بیشترین شانس انتخاب شدن را داراست',
 ' ایران تابعیت دو گانه را به رسمیت نمی\u200cشناسد',
 ' ملاقات با وزیر امورخارجه کوبا و معاون ریاست جمهوری کوبا نیز بخشی از برنامه\u200cهای این سفر است',
 ' در مورد تلفات احتمالی گزارشی نرسیده اس

In [78]:
def remove_puncs(sentence):
  for punct in string.punctuation:
    sentence = sentence.replace(punct, ' ')
  return sentence

In [79]:
def scramble(sent):
    words = sent.split()
    random.shuffle(words)
    return ' '.join(words)

In [80]:
sents_short = [remove_puncs(s) for s in sents_short]

In [81]:
sents_short = sents_short[:5000]

In [82]:
t = Tokenizer(oov_token='<UNK>')
t.fit_on_texts(sents_short)
t.word_index['<PAD>'] = 0

In [83]:
def create_shuffles(sent_list):
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, 6):
      comb_set.add(scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [84]:
pairs = create_shuffles(sents_short)

In [85]:
original = []
shuffled = []
for pair in pairs:
  for p in pair[1]:
    original.append(pair[0])
    shuffled.append(p)

In [86]:
df_dict = {'Original': original, 'Shuffled': shuffled}
df = pd.DataFrame(df_dict)
df

,Original,Shuffled
0,ترکیه که یک کشور اسلامی است مدتهاست که با اسر...,که ترکیه اسلامی که دارد اسرائیل و است یک تجاری...
1,ترکیه که یک کشور اسلامی است مدتهاست که با اسر...,است نزدیک که یک نظامی مناسبات با ترکیه دارد اس...
2,ترکیه که یک کشور اسلامی است مدتهاست که با اسر...,نزدیک کشور مدتهاست که اسلامی با نظامی که ترکیه...
3,ترکیه که یک کشور اسلامی است مدتهاست که با اسر...,که دارد و یک مناسبات ترکیه است اسلامی اسرائیل ...
4,ترکیه که یک کشور اسلامی است مدتهاست که با اسر...,که نظامی اسرائیل یک تجاری مناسبات مدتهاست کشور...
...,...,...
29067,نیروهای آمریکا نیزدراوائل همین هفته درجریان ب...,کشتند برخوردی را درحومه نیزدراوائل شورشی آمریک...
29068,نیروهای آمریکا نیزدراوائل همین هفته درجریان ب...,۲۴ دست کم کشتند نیروهای نیزدراوائل درحومه درجر...
29069,نیروهای آمریکا نیزدراوائل همین هفته درجریان ب...,کم کشتند هفته ۲۴ بغداد، نیزدراوائل شورشی همین ...
29070,نیروهای آمریکا نیزدراوائل همین هفته درجریان ب...,کم نیروهای درحومه برخوردی شورشی را هفته ۲۴ آمر...


In [87]:
dataset = df['Shuffled'].values
labels  = df['Original'].values

In [88]:
COUNT = 5000
train_data   = dataset[:COUNT]
train_labels = labels[:COUNT]

test_data   = dataset[COUNT:COUNT+1000]
test_labels = labels[COUNT:COUNT+1000]

In [89]:
train_sequences = t.texts_to_sequences(train_data)
test_sequences = t.texts_to_sequences(test_data)

tr_label_sequences = t.texts_to_sequences(train_labels)
ts_label_sequences = t.texts_to_sequences(test_labels)

In [90]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=10197
Number of Documents=5000


In [91]:
MAX_SEQUENCE_LENGTH = 200

# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

y_train = sequence.pad_sequences(tr_label_sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_test = sequence.pad_sequences(ts_label_sequences, maxlen=MAX_SEQUENCE_LENGTH)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(5000, 200) (1000, 200)
(5000, 200) (1000, 200)


##**Neural Net**

In [113]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 9.2 MB/s 


In [132]:
VOCAB_SIZE = len(t.word_index)
EMBED_SIZE = 200

In [133]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, Embedding, Dense
from tensorflow.keras.models import Sequential

model = Sequential([
        keras.layers.Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH),
        keras.layers.LSTM(128, recurrent_dropout=0.2, dropout=0.2, return_sequences=True),
        keras.layers.LSTM(128, recurrent_dropout=0.1, dropout=0.1, return_sequences=True),
        keras.layers.Dense(VOCAB_SIZE)      
])

adam = keras.optimizers.Adam(learning_rate=0.01, clipvalue=0.5)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 200, 200)          2039400   
                                                                 
 lstm_38 (LSTM)              (None, 200, 128)          168448    
                                                                 
 lstm_39 (LSTM)              (None, 200, 128)          131584    
                                                                 
 dense_18 (Dense)            (None, 200, 10197)        1315413   
                                                                 
Total params: 3,654,845
Trainable params: 3,654,845
Non-trainable params: 0
_________________________________________________________________


In [134]:
model.fit(X_train, y_train, 
          validation_split=0.1,
          epochs=4, 
          batch_size=16)

Epoch 1/4
282/282 [==============================] - 456s 2s/step - loss: 1.1978 - accuracy: 0.9279 - val_loss: 1.0974 - val_accuracy: 0.9320
Epoch 2/4
282/282 [==============================] - 446s 2s/step - loss: 1.1108 - accuracy: 0.9311 - val_loss: 1.0974 - val_accuracy: 0.9320
Epoch 3/4
282/282 [==============================] - 445s 2s/step - loss: 1.1108 - accuracy: 0.9311 - val_loss: 1.0974 - val_accuracy: 0.9320
Epoch 4/4
282/282 [==============================] - 445s 2s/step - loss: 1.1108 - accuracy: 0.9311 - val_loss: 1.0974 - val_accuracy: 0.9320


In [136]:
y_pred = model.predict(X_test[:1])

In [147]:
X_test.shape, y_test.shape

((1000, 200), (1000, 200))

In [149]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=16)
print("test loss, test acc:", results)

Evaluate on test data
63/63 [==============================] - 8s 132ms/step - loss: 1.1456 - accuracy: 0.9290
test loss, test acc: [1.145646572113037, 0.9289849996566772]


In [159]:
print('Accuracy on train data:', results[1]*100)

Accuracy on train data: 92.89849996566772


**So our model got a 92% accuracy on test data**